<a href="https://colab.research.google.com/github/byunsy/quantitative-momentum/blob/main/Quantitative_Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Momentum Strategy


## Import Necessary Modules

In [1]:
import numpy as np 
import pandas as pd 
import requests 
import math 

In [2]:
from google.colab import files
uploaded = files.upload()

## Attain S&P Stock Listing

Get a list of all the companies in the S&P 500. 

In [3]:
sp500 = pd.read_csv('sp_500_stocks.csv')
sp500

,Symbol
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## API Call

We first need to have a test api token to use IEX Cloud APIs (This will remain private). You can receive sandbox Text APIs from the IEX Cloud API website. 

In [4]:
from iex_api import IEX_CLOUD_API_TOKEN

In [5]:
# To take an example of what we get from IEX Cloud, we will take Microsoft
symbol='MSFT'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
ms_data = requests.get(api_url).json()

ms_data

{'avg10Volume': 32210965,
 'avg30Volume': 27926157,
 'beta': 1.1674097327656912,
 'companyName': 'Microsoft Corporation',
 'day200MovingAvg': 221.59,
 'day30ChangePercent': 0.02307254762200594,
 'day50MovingAvg': 225.87,
 'day5ChangePercent': 0.06446413100950417,
 'dividendYield': 0.009508437141495909,
 'employees': 0,
 'exDividendDate': '2020-11-06',
 'float': 0,
 'marketcap': 1764875578239,
 'maxChangePercent': 8.740028483584293,
 'month1ChangePercent': 0.009061307537760202,
 'month3ChangePercent': 0.05682204525359558,
 'month6ChangePercent': 0.07551647899833484,
 'nextDividendDate': '2021-02-06',
 'nextEarningsDate': '0',
 'peRatio': 36.82188662232647,
 'sharesOutstanding': 7648792275,
 'ttmDividendRate': 2.1122219563584808,
 'ttmEPS': 6.5,
 'week52change': 0.3878217139904963,
 'week52high': 242.42,
 'week52low': 138.08,
 'year1ChangePercent': 0.38604758416241,
 'year2ChangePercent': 1.2375391058000553,
 'year5ChangePercent': 3.8122036832710737,
 'ytdChangePercent': 0.01600076406897

We can now get specific information about our data using indices. 

In [6]:
print("Year 1 Percentage Change:", ms_data['year1ChangePercent'])

Year 1 Percentage Change: 0.38604758416241


**NOTE:**

Since we are using sandbox test APIs, the values returned are not real. 

## Data Preprocessing


We will now transfer our attained data into a data frame. 

In [7]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
symbol_batch = list(chunks(sp500['Symbol'], 100))
symbol_strings = []

for batch in symbol_batch:
    symbol_strings.append(','.join(batch))

df_columns = ['Symbol', 'Price', 'One-Year Price Return']
df = pd.DataFrame(columns=df_columns)

for symbol_string in symbol_strings:

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        df = df.append(pd.Series([symbol, data[symbol]['quote']['latestPrice'], 
                                  data[symbol]['stats']['year1ChangePercent']], 
                                  index=df_columns), ignore_index = True)
        
df

,Symbol,Price,One-Year Price Return
0,A,132.05,0.413691
1,AAL,16.10,-0.437299
2,AAP,166.63,0.11934
3,AAPL,140.25,0.800867
4,ABBV,112.93,0.355596
...,...,...,...
500,YUM,111.65,0.0262247
501,ZBH,167.34,0.077872
502,ZBRA,415.17,0.635136
503,ZION,48.16,0.0449061


## 50 Highgest-momentum stocks in the S&P 500

In [9]:
df_sorted = df.sort_values('One-Year Price Return', ascending=False)
df_sorted = df_sorted[:51]

df_sorted.reset_index(drop=True, inplace=True)
df_sorted

,Symbol,Price,One-Year Price Return
0,CARR,40.35,2.3176
1,FCX,31.65,1.54195
2,LB,47.53,1.33453
3,ALB,174.74,1.22839
4,PYPL,261.00,1.20907
5,NVDA,551.50,1.19972
6,ALGN,534.78,0.950922
7,PWR,79.64,0.928609
8,WST,304.90,0.927295
9,ABMD,361.10,0.923737


## Momentum Strategy

In [10]:
hqm_columns = ['Symbol', 'Price', 'HQM Score', 'Num Shares to Purchase', 
               '1-Year Price Ret', '1-Year Return Per',
               '6-Month Price Ret','6-Month Return Per',
               '3-Month Price Ret','3-Month Return Per',
               '1-Month Price Ret','1-Month Return Per']

In [11]:
hqm_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(pd.Series([symbol, data[symbol]['quote']['latestPrice'],'N/A','N/A',
                                          data[symbol]['stats']['year1ChangePercent'],'N/A',
                                          data[symbol]['stats']['month6ChangePercent'],'N/A',
                                          data[symbol]['stats']['month3ChangePercent'],'N/A',
                                          data[symbol]['stats']['month1ChangePercent'],'N/A'], 
                                          index = hqm_columns), ignore_index = True)
        
hqm_df

,Symbol,Price,HQM Score,Num Shares to Purchase,1-Year Price Ret,1-Year Return Per,6-Month Price Ret,6-Month Return Per,3-Month Price Ret,3-Month Return Per,1-Month Price Ret,1-Month Return Per
0,A,131.07,N/A,N/A,0.415287,N/A,0.318655,N/A,0.190218,N/A,0.0790139,N/A
1,AAL,16.20,N/A,N/A,-0.42312,N/A,0.408777,N/A,0.204602,N/A,0.0222724,N/A
2,AAP,169.30,N/A,N/A,0.121053,N/A,0.10234,N/A,0.0745867,N/A,0.0284305,N/A
3,AAPL,142.40,N/A,N/A,0.786767,N/A,0.435121,N/A,0.207058,N/A,0.0551683,N/A
4,ABBV,112.71,N/A,N/A,0.342353,N/A,0.169526,N/A,0.330831,N/A,0.0953878,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.26,N/A,N/A,0.0255929,N/A,0.159432,N/A,0.0778504,N/A,-0.00293935,N/A
501,ZBH,163.78,N/A,N/A,0.0786678,N/A,0.220046,N/A,0.133638,N/A,0.0717818,N/A
502,ZBRA,424.55,N/A,N/A,0.616283,N/A,0.476437,N/A,0.369919,N/A,0.0768103,N/A
503,ZION,49.41,N/A,N/A,0.0457573,N/A,0.484793,N/A,0.526381,N/A,0.148069,N/A


In [12]:
from scipy import stats

time_periods = ['1-Year','6-Month','3-Month','1-Month']

for row in hqm_df.index:
    for time_period in time_periods:
        if hqm_df.loc[row, f'{time_period} Price Ret'] == None:
            hqm_df.loc[row, f'{time_period} Price Ret'] = 0


for row in hqm_df.index:
    for time_period in time_periods:

        hqm_df.loc[row, f'{time_period} Return Per'] = stats.percentileofscore(a=hqm_df[f'{time_period} Price Ret'], 
                                                                               score=hqm_df.loc[row, f'{time_period} Price Ret'])

# Print the entire DataFrame    
hqm_df

,Symbol,Price,HQM Score,Num Shares to Purchase,1-Year Price Ret,1-Year Return Per,6-Month Price Ret,6-Month Return Per,3-Month Price Ret,3-Month Return Per,1-Month Price Ret,1-Month Return Per
0,A,131.07,N/A,N/A,0.415287,86.9307,0.318655,68.5149,0.190218,63.9604,0.0790139,69.703
1,AAL,16.20,N/A,N/A,-0.42312,1.58416,0.408777,78.2178,0.204602,66.5347,0.0222724,40.198
2,AAP,169.30,N/A,N/A,0.121053,55.6436,0.10234,30.6931,0.0745867,35.2475,0.0284305,43.5644
3,AAPL,142.40,N/A,N/A,0.786767,96.6337,0.435121,81.5842,0.207058,66.9307,0.0551683,59.4059
4,ABBV,112.71,N/A,N/A,0.342353,81.1881,0.169526,45.1485,0.330831,81.7822,0.0953878,74.6535
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.26,N/A,N/A,0.0255929,40.5941,0.159432,42.7723,0.0778504,36.2376,-0.00293935,24.7525
501,ZBH,163.78,N/A,N/A,0.0786678,49.3069,0.220046,53.4653,0.133638,51.0891,0.0717818,66.9307
502,ZBRA,424.55,N/A,N/A,0.616283,93.4653,0.476437,85.1485,0.369919,85.3465,0.0768103,68.7129
503,ZION,49.41,N/A,N/A,0.0457573,43.9604,0.484793,86.1386,0.526381,94.6535,0.148069,88.9109


## HQM (High Quality Momentum) Score

In [13]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []

    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Per'])

    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [14]:
hqm_df

,Symbol,Price,HQM Score,Num Shares to Purchase,1-Year Price Ret,1-Year Return Per,6-Month Price Ret,6-Month Return Per,3-Month Price Ret,3-Month Return Per,1-Month Price Ret,1-Month Return Per
0,A,131.07,72.2772,N/A,0.415287,86.9307,0.318655,68.5149,0.190218,63.9604,0.0790139,69.703
1,AAL,16.20,46.6337,N/A,-0.42312,1.58416,0.408777,78.2178,0.204602,66.5347,0.0222724,40.198
2,AAP,169.30,41.2871,N/A,0.121053,55.6436,0.10234,30.6931,0.0745867,35.2475,0.0284305,43.5644
3,AAPL,142.40,76.1386,N/A,0.786767,96.6337,0.435121,81.5842,0.207058,66.9307,0.0551683,59.4059
4,ABBV,112.71,70.6931,N/A,0.342353,81.1881,0.169526,45.1485,0.330831,81.7822,0.0953878,74.6535
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.26,36.0891,N/A,0.0255929,40.5941,0.159432,42.7723,0.0778504,36.2376,-0.00293935,24.7525
501,ZBH,163.78,55.198,N/A,0.0786678,49.3069,0.220046,53.4653,0.133638,51.0891,0.0717818,66.9307
502,ZBRA,424.55,83.1683,N/A,0.616283,93.4653,0.476437,85.1485,0.369919,85.3465,0.0768103,68.7129
503,ZION,49.41,78.4158,N/A,0.0457573,43.9604,0.484793,86.1386,0.526381,94.6535,0.148069,88.9109


In [24]:
hqm_df_sorted = hqm_df.sort_values(by='HQM Score', ascending=False)
hqm_df_sorted = hqm_df_sorted[:51]

hqm_df_sorted.reset_index(inplace=True)
hqm_df_sorted

,index,Symbol,Price,HQM Score,Num Shares to Purchase,1-Year Price Ret,1-Year Return Per,6-Month Price Ret,6-Month Return Per,3-Month Price Ret,3-Month Return Per,1-Month Price Ret,1-Month Return Per
0,179,FCX,32.00,98.6634,N/A,1.48296,99.802,1.24172,99.604,0.693574,97.8218,0.25982,97.4257
1,410,SIVB,495.93,98.5644,N/A,0.909606,98.2178,1.21562,99.4059,0.692271,97.6238,0.314897,99.0099
2,23,ALB,181.90,97.6733,N/A,1.18753,99.0099,1.02198,98.4158,0.846465,98.8119,0.193894,94.4554
3,203,GM,56.00,96.4851,N/A,0.623344,94.0594,1.15654,99.2079,0.49362,93.2673,0.367984,99.4059
4,29,AMAT,108.84,95.7921,N/A,0.684917,94.6535,0.681981,93.4653,0.736495,98.0198,0.245754,97.0297
5,314,MOS,29.73,94.8515,N/A,0.437135,87.7228,1.12441,98.8119,0.525109,94.4554,0.306922,98.4158
6,288,LRCX,566.64,94.6535,N/A,0.844903,97.6238,0.572629,90.6931,0.586247,96.6337,0.183816,93.6634
7,275,LB,47.16,94.1089,N/A,1.35784,99.604,1.33956,99.802,0.336268,82.1782,0.194413,94.8515
8,266,KLAC,306.66,93.2178,N/A,0.75504,96.0396,0.515341,88.1188,0.528263,94.8515,0.187669,93.8614
9,324,MU,84.21,91.4851,N/A,0.394863,85.7426,0.604057,92.0792,0.532754,95.0495,0.172931,93.0693


In [29]:
PORTFOLIO_SIZE = 100000
position_size = float(PORTFOLIO_SIZE) / len(hqm_df_sorted.index)
print(position_size)

1960.7843137254902


In [30]:
for i in range(0, len(hqm_df_sorted['Symbol'])):
    hqm_df_sorted.loc[i, 'Num Shares to Purchase'] = math.floor(position_size / hqm_df_sorted['Price'][i])

hqm_df_sorted

,index,Symbol,Price,HQM Score,Num Shares to Purchase,1-Year Price Ret,1-Year Return Per,6-Month Price Ret,6-Month Return Per,3-Month Price Ret,3-Month Return Per,1-Month Price Ret,1-Month Return Per
0,179,FCX,32.00,98.6634,61,1.48296,99.802,1.24172,99.604,0.693574,97.8218,0.25982,97.4257
1,410,SIVB,495.93,98.5644,3,0.909606,98.2178,1.21562,99.4059,0.692271,97.6238,0.314897,99.0099
2,23,ALB,181.90,97.6733,10,1.18753,99.0099,1.02198,98.4158,0.846465,98.8119,0.193894,94.4554
3,203,GM,56.00,96.4851,35,0.623344,94.0594,1.15654,99.2079,0.49362,93.2673,0.367984,99.4059
4,29,AMAT,108.84,95.7921,18,0.684917,94.6535,0.681981,93.4653,0.736495,98.0198,0.245754,97.0297
5,314,MOS,29.73,94.8515,65,0.437135,87.7228,1.12441,98.8119,0.525109,94.4554,0.306922,98.4158
6,288,LRCX,566.64,94.6535,3,0.844903,97.6238,0.572629,90.6931,0.586247,96.6337,0.183816,93.6634
7,275,LB,47.16,94.1089,41,1.35784,99.604,1.33956,99.802,0.336268,82.1782,0.194413,94.8515
8,266,KLAC,306.66,93.2178,6,0.75504,96.0396,0.515341,88.1188,0.528263,94.8515,0.187669,93.8614
9,324,MU,84.21,91.4851,23,0.394863,85.7426,0.604057,92.0792,0.532754,95.0495,0.172931,93.0693
